## 데이터 준비와 전처리 하기

In [ ]:
import os
import pandas as pd
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

In [3]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [9]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
# 평점과 무비 데이터프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,ratings,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [11]:
# 사용하지 않는 컬럼 제거
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,ratings,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


In [13]:
## 데이터 분석하기

In [14]:
# 사용자 수와 영화의 수 구하기
num_watcher = ratings['user_id'].unique()
num_movies = ratings['movie_id'].unique()

print('number of watchers:',num_watcher)
print('number of movies:',num_movies)

number of watchers: [   1    2    3 ... 6038 6039 6040]
number of movies: [1193  661  914 ...  690 2909 1360]


In [15]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [16]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [17]:
## 내가 선호하는 영화

In [22]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('princess', regex=False)]

,movie_id,title,genre
259,262,"Little Princess, A (1995)",Children's|Drama
310,313,"Swan Princess, The (1994)",Animation|Children's
576,580,Princess Caraboo (1994),Drama
905,917,"Little Princess, The (1939)",Children's|Drama
1179,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
2931,3000,"Princess Mononoke, The (Mononoke Hime) (1997)",Action|Adventure|Animation


In [31]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = [262, 313, 576 , 905, 1197]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# '1234'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['1234']*5, 'movie_id': my_favorite_id, 'rating': [5]*5, 'title': my_favorite_title})

my_movielist

,user_id,movie_id,rating,title
0,1234,262,5,"Little Princess, A (1995)"
1,1234,313,5,"Swan Princess, The (1994)"
2,1234,576,5,Fausto (1993)
3,1234,905,5,It Happened One Night (1934)
4,1234,1197,5,"Princess Bride, The (1987)"


In [32]:
if not ratings.isin({'user_id':['1234']})['user_id'].any():  
    ratings = ratings.append(my_movielist, ignore_index=True)   

ratings.tail(7)     

,user_id,movie_id,ratings,title,rating
836476,6040,1096,4.0,Sophie's Choice (1982),NaN
836477,6040,1097,4.0,E.T. the Extra-Terrestrial (1982),NaN
836478,1234,262,NaN,"Little Princess, A (1995)",5.0
836479,1234,313,NaN,"Swan Princess, The (1994)",5.0
836480,1234,576,NaN,Fausto (1993),5.0
836481,1234,905,NaN,It Happened One Night (1934),5.0
836482,1234,1197,NaN,"Princess Bride, The (1987)",5.0


In [33]:
## csr matrix

In [34]:
num_watcher = ratings['user_id'].unique()
num_movies = ratings['title'].unique()

In [40]:
# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(num_watcher)}
movie_to_idx = {v:k for k,v in enumerate(num_movies)}


temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column 인덱싱 성공!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id 인덱싱 실패')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column 인덱싱 성공!!')
    ratings['title'] = temp_movie_data
else:
    print('title column 인덱싱 실패')

ratings

user_id 인덱싱 실패
title column 인덱싱 실패


,user_id,movie_id,ratings,title,rating
0,0,1193,5.0,0,NaN
1,0,661,3.0,1,NaN
2,0,914,3.0,2,NaN
3,0,3408,4.0,3,NaN
4,0,2355,5.0,4,NaN
...,...,...,...,...,...
836478,6039,262,NaN,960,5.0
836479,6039,313,NaN,947,5.0
836480,6039,576,NaN,3090,5.0
836481,6039,905,NaN,1284,5.0


In [42]:
num_watcher = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()

# CSR Matrix 생성
csr_data = csr_matrix((ratings['rating'], (ratings.movie_id, ratings.title)), shape= (num_watcher, num_movies))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.float64'>'
	with 3628 stored elements in Compressed Sparse Row format>

In [43]:
## AlternatingLeastSquares 모델 훈련

In [62]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

ModelFitError: NaN encountered in factors

In [45]:
ratings

,user_id,movie_id,ratings,title,rating
0,0,1193,5.0,0,NaN
1,0,661,3.0,1,NaN
2,0,914,3.0,2,NaN
3,0,3408,4.0,3,NaN
4,0,2355,5.0,4,NaN
...,...,...,...,...,...
836478,6039,262,NaN,960,5.0
836479,6039,313,NaN,947,5.0
836480,6039,576,NaN,3090,5.0
836481,6039,905,NaN,1284,5.0


In [53]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
jm, SS = user_to_idx['1234'], movie_to_idx['Matrix, The (1999)']
jm_vector, SS_vector = als_model.user_factors[hyunji], als_model.item_factors[SS]

jm_vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [54]:
SS_vector

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [55]:
np.dot(jm_vector, SS_vector)

nan

In [50]:
## 영화 추천

In [60]:
favorite_movie = 'Little Princess, A (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   

[(1203, nan),
 (1192, nan),
 (1217, nan),
 (1206, nan),
 (1207, nan),
 (1208, nan),
 (1209, nan),
 (1210, nan),
 (1211, nan),
 (1212, nan),
 (1213, nan),
 (1214, nan),
 (1215, nan),
 (1216, nan),
 (3627, nan)]

In [61]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Something to Talk About (1995)',
 'How Stella Got Her Groove Back (1998)',
 'Blue Chips (1994)',
 'Hope Floats (1998)',
 'Milk Money (1994)',
 'Miami Rhapsody (1995)',
 'Oxygen (1999)',
 'Blame It on Rio (1984)',
 'Fools Rush In (1997)',
 'Tin Cup (1996)',
 'Nine 1/2 Weeks (1986)',
 'Random Hearts (1999)',
 'Metropolitan (1990)',
 'Client, The (1994)',
 'Identification of a Woman (Identificazione di una donna) (1982)']

### 회고
- 이번 노드에는 아쉬운 점이 많았다. 데이터프레임의 사용법을 익히지 못했고 중간에 에러가 뜨고, nan 이라고 뜨는데 원인을 찾지 못했다.결과를 얻지 못하는거에 아쉽다. 그리고 스스로에게 부족함을 많이 느꼈다,
- 앞으로 노드는 밀리지않고 당일 노드는 당일에 하도록 하여야겠다. 밀리다 보니 노드에 대한 집중도가 떨어지고 급하게 하게되어 생각할 시간을 얻지 못하고 악순환이 진행되고 있다.
- 데이터 프레임에 대한 공부를 해야겠다. 
- 그리고 앞에 fun, ex 를 복습을 해야겠다. (너무 하고싶은데 시간이 부족하다)
- 열심히 해야겠다..... 잠자는 시간을 줄여야겠다 ㅜㅜ